## Day 1

In part 2, increment of sum of three consecutive numbers is determined from the sliding window of **four** elements. The head of the four elements is going out, and the tail element is incomming.  

In [14]:
# day 1, part 1

import itertools
# python 3.10 introduces itertools.pairwise

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for a, b in itertools.pairwise(xs) if a < b)
    print(count)

1752


In [15]:
# day 1, part 2

import itertools
import collections
from typing import Iterable, Generator, Any

# https://docs.python.org/3/library/itertools.html#itertools-recipes
def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) -> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(itertools.islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

with open("./2021/day01_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    count = sum(1 for head, _, _, tail in sliding_window(xs, 4) if head < tail)
    print(count)

1781


## Day 2

Discuss change in submarine's position both horizontally and vertically (as depth). The part 2 introduces "aim" as an additional variable.

In [9]:
from typing import Iterable

def day02_part1(xs: Iterable[tuple[str, int]]) -> int:
    h, d = 0, 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
        elif cmd == "down":
            d += amount
        elif cmd == "up":
            d -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [10]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part1(xs)
    print(res)

1804520


In [20]:
def day02_part2(xs: Iterable[tuple[str, int]]) -> int:
    h = 0
    d = 0
    aim = 0
    for cmd, amount in xs:
        if cmd == "forward":
            h += amount
            d += amount * aim
        elif cmd == "down":
            aim += amount
        elif cmd == "up":
            aim -= amount
        else:
            raise ValueError(f"Unknown command: {cmd} {amount}")
    return h * d

In [21]:
with open("./2021/day02_input.txt") as f:
    xs = [s.split() for s in f.readlines() if s.strip()]
    xs = [(cmd, int(w)) for cmd, w in xs]
    res = day02_part2(xs)
    print(res)


1971095320


## Day 3: Binary manipulations

In [41]:
import collections
from typing import Iterable


def day03_part1(content: str) -> int:

    def most_common(xs: Iterable[str]) -> str:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return str(int(cnt['0'] <= cnt['1']))
    
    lines = [list(line.strip()) for line in content.split() if line.strip()]
    gamma = "".join(most_common(seq) for seq in zip(*lines))
    gamma = int(gamma, 2)
    bin_length = len(lines[0])
    epsilon = ((1 << bin_length) - 1) ^ gamma
    print(f"{(gamma, epsilon) = }")
    return gamma * epsilon


In [42]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part1(s)

(gamma, epsilon) = (22, 9)


198

In [43]:
with open("./2021/day03_input.txt") as f:
    res = day03_part1(f.read())
    print(res)

(gamma, epsilon) = (190, 3905)
741950


In [44]:
import numpy as np
from typing import Iterable
import collections

def day03_part2(content: str) -> int:

    def most_common(xs: Iterable[int]) -> int:
        """Assert xs contains either 0 or 1"""
        cnt = collections.Counter(xs)
        return int(cnt[0] <= cnt[1])
    

    def _rating(zs, n_digits, cmp) -> int:
        xs = np.copy(zs)
        
        i = n_digits - 1
        while len(xs) > 1:
            digits = (xs >> i) & 1
            criteria = cmp(digits, most_common(digits))
            xs = xs[criteria]
            i -= 1
        return xs[0]

    def oxygen_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.equal)

    def co2_rating(zs, n_digits) -> int:
        return _rating(zs, n_digits, np.not_equal)
    
    lines = [line.strip() for line in content.split() if line.strip()]
    xs = np.array([int(line, 2) for line in lines])

    n_digits = len(lines[0])
    oxygen = oxygen_rating(xs, n_digits)
    co2 = co2_rating(xs, n_digits)
    print(f"{(oxygen, co2) = }")
    return oxygen * co2


In [45]:
s = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
"""

day03_part2(s)

(oxygen, co2) = (23, 10)


230

In [46]:
with open("./2021/day03_input.txt") as f:
    res = day03_part2(f.read())
    print(res)

(oxygen, co2) = (282, 3205)
903810


## Day 4: 5 x 5 Bingo

In [5]:
def day04_parse(content: str) -> tuple[list[int], list[list[list[int]]]]:
    """Return integer sequence AND a list of 5x5 bingos"""
    chunks = [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]
    xs = [int(s) for s in chunks[0].split(",") if s.strip()]
    dominos = [
        [
            [int(s) for s in line.split()]
            for line in chunk.split("\n") if line.strip()
        ] 
        for chunk in chunks[1:]
    ]
    return xs, dominos



In [9]:
s = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""

calls, dominos = day04_parse(s)
dominos

[[[22, 13, 17, 11, 0],
  [8, 2, 23, 4, 24],
  [21, 9, 14, 16, 7],
  [6, 10, 3, 18, 5],
  [1, 12, 20, 15, 19]],
 [[3, 15, 0, 2, 22],
  [9, 18, 13, 17, 5],
  [19, 8, 7, 25, 23],
  [20, 11, 10, 24, 4],
  [14, 21, 16, 12, 6]],
 [[14, 21, 17, 24, 4],
  [10, 16, 15, 9, 19],
  [18, 8, 23, 26, 20],
  [22, 11, 13, 6, 5],
  [2, 0, 12, 3, 7]]]